# Исследование надежности заемщиков


**Цель исследования:** На основе статистики о платёжеспособности клиентов исследовать, влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.

## Изучим общую информацию о данных.

In [1]:
import pandas as pd

data = pd.read_csv('')

In [2]:
# изучим первые 20 строк
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# выведем общую информацию
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных.

### Удаление пропусков.

Посчитаем пропуски в столбцах.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

Исправим отрицательное количество дней трудового стажа в столбце `days_employed` с помощью метода `abs()`.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
round(data.groupby('income_type')['days_employed'].agg('median'))

income_type
безработный        366414.0
в декрете            3297.0
госслужащий          2689.0
компаньон            1547.0
пенсионер          365213.0
предприниматель       521.0
сотрудник            1574.0
студент               579.0
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, Оставим как есть. Тем более данный столбец не потребуется для исследования.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
# проверим
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()
    
# проверим
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных.

In [11]:
# заменим вещественный тип данных в столбце `total_income` на целочисленный
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем к нижнему регистру.

In [12]:
data['education'] = data['education'].str.lower()

Посчитаем явные дубликаты, и очистим при необходимости.

In [13]:
data.duplicated().sum()

71

In [14]:
data = data.drop_duplicates()

### Категоризация данных.

Создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [15]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [16]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [17]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [18]:
# по сути, формируем категории по маске
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [19]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные.

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [20]:
#создаём сводную таблицу для наглядного предоставления/рассчета требуемых данных
#.droplevel(level=1, axis=1), удаляем лишнее в названии столбцов
#в таблице сразу переименуем столбцы для более удобного восприятия.
debt = data.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean']).droplevel(level=1, axis=1)\
.rename(columns={'count':'Кредиты','sum':'Просрочено','mean':'Доля просроченных'}).rename_axis('Кол-во детей') 

#выводим целевую таблицу
debt


,Кредиты,Просрочено,Доля просроченных
Кол-во детей,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод:** По полученной таблице можно сделать вывод, самые надёжные заёмщики - бездетные и с тремя детьми. А большинство просрочек, в свою очередь, у заёмщиков с 1 и 2 детьми.
По заёмщикам с 4 и 5 детьми сложно сделать вывод, так как по ним маленькая выборка.

In [21]:
#смотрим следующие поставленные вопросы и делаем вывод - они типовые. Значит удобнее написать функцию.
#обратим внимание на структуру data - данные из следующих поставленных вопросов уже категоризированы
#анализ следует проводить именно по категориям

def debt_percent(category):
    debt = data.pivot_table(index=category, values='debt', aggfunc=['count', 'sum', 'mean']).droplevel(level=1, axis=1)\
    .rename(columns={'count':'Кредиты','sum':'Просрочено','mean':'Доля просроченных'})
    return debt

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
debt_percent('family_status').rename_axis('Семейное положение') 
#вызываем функцию, в качестве аргумента передаём название столбца, по которому проводим анализ
#переименуем столбец для удобства

,Кредиты,Просрочено,Доля просроченных
Семейное положение,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606


**Вывод:** Заёмщики, состоявщие/состоящие в законных отношениях, являются самыми надёжными.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
temp = debt_percent('total_income_category').rename_axis('Категория заёмщика')
#чтобы нашему заказчику было проще, добавим столбец с указанием дохода
temp.insert(0,'Доход',['свыше 1000000','200001–1000000','50001–200000','30001–50000','0–30000'])
temp

,Доход,Кредиты,Просрочено,Доля просроченных
Категория заёмщика,,,,
A,свыше 1000000,25,2,0.080000
B,200001–1000000,5014,354,0.070602
C,50001–200000,15921,1353,0.084982
D,30001–50000,349,21,0.060172
E,0–30000,22,2,0.090909


**Вывод:** Самые надежные заёмщики - категории D, самые ненадежные - C. По категориям E и A выводы неоднозначны, т.к. слишком мало случаев.

#### Как разные цели кредита влияют на его возврат в срок?

In [24]:
debt_percent('purpose_category').rename_axis('Цель кредитования').sort_values('Доля просроченных')

,Кредиты,Просрочено,Доля просроченных
Цель кредитования,,,
операции с недвижимостью,10751,780,0.072551
проведение свадьбы,2313,183,0.079118
получение образования,3988,369,0.092528
операции с автомобилем,4279,400,0.093480


**Вывод:** Здесь же самые ответственные заёмщики те, кто берёт кредит на недвижимость. Кроме того, таких операций больше всего.
Те, кто берёт кредит на свадьбу, оказались на втором месте, что косвенно подтверждает гипотезу 2.6.2
А вот получение образования и автокредиты оказались на последнем месте.

#### Возможные причины появления пропусков в исходных данных.

В исходном датафрейме были пропуски в следующих столбцах

days_employed     19351 non-null  float64<br>total_income      19351 non-null  float64

вероятнее всего заёмщики, у которых не указаны эти данные либо безработные, либо отказались их указывать при внесении в базу.

### Общий вывод.

Было проведено исследование надежности заёмщиков.
В ходе предварительной обработки данных выяснено, что столбец о рабочих днях содержит много отрицательных значений, 
вероятно стоит исправить алгоритм внесения этих значений.

<b>Также в ходе предобработки были категоризированы данные по доходу потенциального заёмщика (всего выделено 5 категорий):</b>
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

<b>Выяснены основные цели, на которые берут кредиты:</b>
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

Эти данные в будущем возможно использовать для упрощения/автоматизации заполнения анкеты заёмщика.

<b>По результатам исследования самыми надежными оказались заёмщики, которые:</b>
- не имеют детей, или имеют 3х детей
- состоят/состояли в официально зарегистрированных семейных отношениях
- имеют доход 30000-50000
- берут кредит на свадьбу, либо на операции с недвижимостью

<b>Самыми ненадежными оказались те, кто:</b>
- имеет 1-2 детей
- никогда не состояли в официально зарегистрированных семейных отношениях
- имеют доход 50000-200000
- берут кредит на автомобиль или образование

<b>При этом, самые часто встречающиеся категории заёмщиков:</b>
- бездетные
- женаты/замужем
- имеют доход 50000-200000

<b>Также, самая частая цель кредита - операции с недвижимостью.</b>